In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import sympy as sp
from tqdm import tqdm
import numpy as np
np.set_printoptions(suppress=True)

data = pd.read_csv(r"E:\参加点比赛\数模研赛\data\碳排预测数据.csv")
print(data.columns)
system = data.copy()

d:\Develop\Anaconda3\envs\py38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\Develop\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\Develop\Anaconda3\envs\py38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Index(['时间', '常驻人口总量', '区域生产总值', '一次能源中化石的能源消费量', '一次能源中非化石的能源消费量',
       '二次能源的能源消费量', '化石能源发电的能源消费量', '非化石能源发电的能源消费量', '农林消费部门的碳排放量',
       '工业消费部门的碳排放量', '交通消费部门的碳排放量', '建筑消费部门的碳排放量', '居民生活消费部门的碳排放量',
       '能源供应部门的碳排放量', '碳排放量'],
      dtype='object')


### 计算10-20的能源利用效率和非化石能源消费比重

对2030年的人口数据，设置不同的参数，拟合出不同的方程

In [16]:
data = pd.read_csv(r"E:\参加点比赛\数模研赛\data\碳排预测数据.csv")
system = data.copy()

w_list = np.linspace(4863, 4868, 20)
for i in tqdm(range(len(w_list))):
    w = w_list[i] # 权重
    sign1 = False # 判断碳达峰时间约束
    sign2 = False # 判断预测结果正负约束
    sign3 = False # 判断碳中和约束
    # 假设条件下拟合得到人口、经济与时间的关系式
    ##############################################################################################################################################
    # 对人口做拟合
    # 做多项式拟合
    x = system['时间'].values.tolist()
    P = system['常驻人口总量'].values.tolist()
    x = x + [2030]
    P = P + [w]
    def fit_p_poly(x, a, b, c):
        return a * x**2 + b * x + c
    params, convariance = curve_fit(fit_p_poly, x, P)
    a, b, c = params
    x = sp.symbols('x')
    predict_p_poly = a * x**2 + b * x + c
    # 做线性拟合
    x = system['时间'].values.tolist()
    P = system['常驻人口总量'].values.tolist()
    def fit_p_linear(x, a, b): 
        return a * x + b
    params, convariance = curve_fit(fit_p_linear, x, P)
    a, b = params
    x = sp.symbols('x')
    predict_p_linear = a * x + b 
    # 组合预测
    predict_p = 0.8 * predict_p_linear + 0.2 * predict_p_poly
    # 对经济做拟合：满足给定条件
    x = system['时间'].values.tolist()
    GDP = system['区域生产总值'].values.tolist()
    x = x + [2035, 2060]
    GDP = GDP + [2*GDP[-1], 4*GDP[-1]]
    def fit_gdp(x, a, b): # 拟合函数
        return a * x + b
    params, convariance = curve_fit(fit_gdp, x, GDP) # 将x、GDP放入fit_gdp中拟合
    a, b = params # 获取拟合参数
    x = sp.symbols('x') 
    predict_gdp = a * x + b # 获取拟合方程式


    # 多元线性回归 得到五种能源与人口、经济关系
    ##############################################################################################################################################
    # 选择自变量与应变量
    GDPP = system.loc[:, ['区域生产总值', '常驻人口总量']].values # 自变量
    EV1 = system['一次能源中化石的能源消费量'].values
    EV2 = system['一次能源中非化石的能源消费量'].values
    EV3 = system['二次能源的能源消费量'].values
    EV4 = system['化石能源发电的能源消费量'].values
    EV5 = system['非化石能源发电的能源消费量'].values

    # 求解EV1关于GDP的线性回归方程
    lr_EV1 = LinearRegression()
    lr_EV1.fit(GDPP, EV1)
    coef_EV1 = lr_EV1.coef_
    intercept_EV1 = lr_EV1.intercept_
    R2_EV1 = lr_EV1.score(GDPP, EV1)

    # 求解EV2关于GDP的线性回归方程
    lr_EV2 = LinearRegression()
    lr_EV2.fit(GDPP, EV2)
    coef_EV2 = lr_EV2.coef_
    intercept_EV2 = lr_EV2.intercept_
    R2_EV2 = lr_EV2.score(GDPP, EV2)

    # 求解EV3关于GDP的线性回归方程
    lr_EV3 = LinearRegression()
    lr_EV3.fit(GDPP, EV3)
    coef_EV3 = lr_EV3.coef_
    intercept_EV3 = lr_EV3.intercept_
    R2_EV3 = lr_EV3.score(GDPP, EV3)

    # 求解EV4关于GDP的线性回归方程
    lr_EV4 = LinearRegression()
    lr_EV4.fit(GDPP, EV4)
    coef_EV4 = lr_EV4.coef_
    intercept_EV4 = lr_EV4.intercept_
    R2_EV4 = lr_EV4.score(GDPP, EV4)

    # 求解EV5关于GDP的线性回归方程
    lr_EV5 = LinearRegression()
    lr_EV5.fit(GDPP, EV5)
    coef_EV5 = lr_EV5.coef_
    intercept_EV5 = lr_EV5.intercept_
    R2_EV5 = lr_EV5.score(GDPP, EV5)

    # 多元线性回归 得到六个部门碳排放量与五种能源消费量关系
    ##############################################################################################################################################
    # 选择自变量与应变量
    Eall = system.loc[:, ['一次能源中化石的能源消费量', '一次能源中非化石的能源消费量', '二次能源的能源消费量', '化石能源发电的能源消费量',
        '非化石能源发电的能源消费量']]
    CO2D1 = system['农林消费部门的碳排放量'].values
    CO2D2 = system['工业消费部门的碳排放量'].values
    CO2D3 = system['交通消费部门的碳排放量'].values
    CO2D4 = system['建筑消费部门的碳排放量'].values
    CO2D5 = system['居民生活消费部门的碳排放量'].values
    CO2D6 = system['能源供应部门的碳排放量'].values
    CO2 = system['碳排放量'].values

    # 求解CO2D1关于五种能源的方程
    lr_CO2D1 = LinearRegression()
    lr_CO2D1.fit(Eall, CO2D1)
    coef_CO2D1 = lr_CO2D1.coef_
    intercept_CO2D1 = lr_CO2D1.intercept_
    R2_CO2D1 = lr_CO2D1.score(Eall, CO2D1)

    # 求解CO2D2关于五种能源的方程
    lr_CO2D2 = LinearRegression()
    lr_CO2D2.fit(Eall, CO2D2)
    coef_CO2D2 = lr_CO2D2.coef_
    intercept_CO2D2 = lr_CO2D2.intercept_
    R2_CO2D2 = lr_CO2D2.score(Eall, CO2D2)

    # 求解CO2D3关于五种能源的方程
    lr_CO2D3 = LinearRegression()
    lr_CO2D3.fit(Eall, CO2D3)
    coef_CO2D3 = lr_CO2D3.coef_
    intercept_CO2D3 = lr_CO2D3.intercept_
    R2_CO2D3 = lr_CO2D3.score(Eall, CO2D3)

    # 求解CO2D4关于五种能源的方程
    lr_CO2D4 = LinearRegression()
    lr_CO2D4.fit(Eall, CO2D4)
    coef_CO2D4 = lr_CO2D4.coef_
    intercept_CO2D4 = lr_CO2D4.intercept_
    R2_CO2D4 = lr_CO2D4.score(Eall, CO2D4)

    # 求解CO2D5关于五种能源的方程
    lr_CO2D5 = LinearRegression()
    lr_CO2D5.fit(Eall, CO2D5)
    coef_CO2D5 = lr_CO2D5.coef_
    intercept_CO2D5 = lr_CO2D5.intercept_
    R2_CO2D5 = lr_CO2D5.score(Eall, CO2D5)

    # 求解CO2D6关于五种能源的方程
    lr_CO2D6 = LinearRegression()
    lr_CO2D6.fit(Eall, CO2D6)
    coef_CO2D6 = lr_CO2D6.coef_
    intercept_CO2D6 = lr_CO2D6.intercept_
    R2_CO2D6 = lr_CO2D6.score(Eall, CO2D6)

    # 计算得到CO2的方程
    ##############################################################################################################################################
    predict_EV1 = coef_EV1[0] * predict_gdp + coef_EV1[1] * predict_p + intercept_EV1
    predict_EV2 = coef_EV2[0] * predict_gdp + coef_EV2[1] * predict_p + intercept_EV2
    predict_EV3 = coef_EV3[0] * predict_gdp + coef_EV3[1] * predict_p + intercept_EV3
    predict_EV4 = coef_EV4[0] * predict_gdp + coef_EV4[1] * predict_p + intercept_EV4
    predict_EV5 = coef_EV5[0] * predict_gdp + coef_EV5[1] * predict_p + intercept_EV5

    predict_CO2D1 = coef_CO2D1[0] * predict_EV1 + \
                    coef_CO2D1[1] * predict_EV2 + \
                    coef_CO2D1[2] * predict_EV3 + \
                    coef_CO2D1[3] * predict_EV4 + \
                    coef_CO2D1[4] * predict_EV5 + intercept_CO2D1

    predict_CO2D2 = coef_CO2D2[0] * predict_EV1 + \
                    coef_CO2D2[1] * predict_EV2 + \
                    coef_CO2D2[2] * predict_EV3 + \
                    coef_CO2D2[3] * predict_EV4 + \
                    coef_CO2D2[4] * predict_EV5 + intercept_CO2D2

    predict_CO2D3 = coef_CO2D3[0] * predict_EV1 + \
                    coef_CO2D3[1] * predict_EV2 + \
                    coef_CO2D3[2] * predict_EV3 + \
                    coef_CO2D3[3] * predict_EV4 + \
                    coef_CO2D3[4] * predict_EV5 + intercept_CO2D3

    predict_CO2D4 = coef_CO2D4[0] * predict_EV1 + \
                    coef_CO2D4[1] * predict_EV2 + \
                    coef_CO2D4[2] * predict_EV3 + \
                    coef_CO2D4[3] * predict_EV4 + \
                    coef_CO2D4[4] * predict_EV5 + intercept_CO2D4

    predict_CO2D5 = coef_CO2D5[0] * predict_EV1 + \
                    coef_CO2D5[1] * predict_EV2 + \
                    coef_CO2D5[2] * predict_EV3 + \
                    coef_CO2D5[3] * predict_EV4 + \
                    coef_CO2D5[4] * predict_EV5 + intercept_CO2D5

    predict_CO2D6 = coef_CO2D6[0] * predict_EV1 + \
                    coef_CO2D6[1] * predict_EV2 + \
                    coef_CO2D6[2] * predict_EV3 + \
                    coef_CO2D6[3] * predict_EV4 + \
                    coef_CO2D6[4] * predict_EV5 + intercept_CO2D6
    predict_CO2 = predict_CO2D1 + predict_CO2D2 + predict_CO2D3 + predict_CO2D4 + predict_CO2D5 + predict_CO2D6
    # 求解碳达峰时间
    predict_CO2_d = sp.diff(predict_CO2, x)
    t1 = sp.solve(predict_CO2_d)
    t1 = int(t1[0])

    # 满足2030年碳达峰条件
    if (t1>2020) & (t1<2050):
        sign1 = True

    # 新建预测结果
    system_pre = pd.DataFrame(columns=system.columns)
    system_pre['时间'] = [i for i in range(2010, 2061)]

    for i in range(system_pre.shape[0]):
        t = system_pre.loc[i, ['时间']][0] # 获取第i个时间
        system_pre.loc[i, ['常驻人口总量']] = predict_p.subs(x, t)
        system_pre.loc[i, ['区域生产总值']] = predict_gdp.subs(x, t)
        system_pre.loc[i, ['一次能源中化石的能源消费量']] = predict_EV1.subs(x, t)
        system_pre.loc[i, ['一次能源中非化石的能源消费量']] = predict_EV2.subs(x, t)
        system_pre.loc[i, ['二次能源的能源消费量']] = predict_EV3.subs(x, t)
        system_pre.loc[i, ['化石能源发电的能源消费量']] = predict_EV4.subs(x, t)
        system_pre.loc[i, ['非化石能源发电的能源消费量']] = predict_EV5.subs(x, t)
        system_pre.loc[i, ['农林消费部门的碳排放量']] = predict_CO2D1.subs(x, t)
        system_pre.loc[i, ['工业消费部门的碳排放量']] = predict_CO2D2.subs(x, t)
        system_pre.loc[i, ['交通消费部门的碳排放量']] = predict_CO2D3.subs(x, t)
        system_pre.loc[i, ['建筑消费部门的碳排放量']] = predict_CO2D4.subs(x, t)
        system_pre.loc[i, ['居民生活消费部门的碳排放量']] = predict_CO2D5.subs(x, t)
        system_pre.loc[i, ['能源供应部门的碳排放量']] = predict_CO2D6.subs(x, t)
        system_pre.loc[i, ['碳排放量']] = predict_CO2.subs(x, t)

    # 判断是否全都为正
    has_negative = (system_pre['碳排放量'] > 0).all()
    if has_negative:
        sign2 = True

    time = [2020, 2025, 2030, 2035, 2050, 2060]
    goal_2 = pd.DataFrame({'时间' : time})
    goal_2['一次能源中化石的能源消费量'] = 99
    goal_2['一次能源中非化石的能源消费量'] = 99
    goal_2['二次能源的能源消费量'] = 99
    goal_2['化石能源发电的能源消费量'] = 99
    goal_2['非化石能源发电的能源消费量'] = 99
    goal_2['碳排放量'] = 99
    for i in time:
        if i == 2020:
            goal_2.loc[goal_2['时间']==i, ['一次能源中化石的能源消费量']] = system.loc[system['时间']==i, ['一次能源中化石的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['一次能源中非化石的能源消费量']] = system.loc[system['时间']==i, ['一次能源中非化石的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['二次能源的能源消费量']] = system.loc[system['时间']==i, ['二次能源的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['化石能源发电的能源消费量']] = system.loc[system['时间']==i, ['化石能源发电的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['非化石能源发电的能源消费量']] = system.loc[system['时间']==i, ['非化石能源发电的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['碳排放量']] = system.loc[system['时间']==i, ['碳排放量']].values[0]
        else:
            goal_2.loc[goal_2['时间']==i, ['一次能源中化石的能源消费量']] = system_pre.loc[system_pre['时间']==i, ['一次能源中化石的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['一次能源中非化石的能源消费量']] = system_pre.loc[system_pre['时间']==i, ['一次能源中非化石的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['二次能源的能源消费量']] = system_pre.loc[system_pre['时间']==i, ['二次能源的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['化石能源发电的能源消费量']] = system_pre.loc[system_pre['时间']==i, ['化石能源发电的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['非化石能源发电的能源消费量']] = system_pre.loc[system_pre['时间']==i, ['非化石能源发电的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['碳排放量']] = system_pre.loc[system_pre['时间']==i, ['碳排放量']].values[0]
    goal_2['能源利用效率'] = goal_2['碳排放量'] / \
                            (goal_2['一次能源中化石的能源消费量'] + \
                            goal_2['一次能源中非化石的能源消费量'] + \
                            goal_2['二次能源的能源消费量'])
    goal_2['非化石能源消费比重'] = (goal_2['一次能源中非化石的能源消费量'] + goal_2['二次能源的能源消费量']) / \
                                (goal_2['一次能源中化石的能源消费量'] + goal_2['一次能源中非化石的能源消费量'] + goal_2['二次能源的能源消费量'] )
    if goal_2.loc[goal_2['时间']==2060, ['非化石能源消费比重']].values[0][0] > 0.8:
        sign3 = True
    
    if sign1 & sign2:
        print(goal_2.loc[goal_2['时间']==2060, ['非化石能源消费比重']].values[0][0])
        print("满足线性规划模型的人口={}碳达峰时间={}".format(w, t1))


  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:00<00:16,  1.17it/s]

-1.66996718947522
满足线性规划模型的人口=4863.0碳达峰时间=2025


 10%|█         | 2/20 [00:01<00:15,  1.13it/s]

-1.67040050572516
满足线性规划模型的人口=4863.263157894737碳达峰时间=2025


...

 85%|████████▌ | 17/20 [00:14<00:02,  1.21it/s]

-1.67693041402027
满足线性规划模型的人口=4867.210526315789碳达峰时间=2026


 90%|█████████ | 18/20 [00:15<00:01,  1.18it/s]

-1.67736776212651
满足线性规划模型的人口=4867.473684210527碳达峰时间=2026


KeyboardInterrupt: 

In [ ]:
np.linspace(7500, 9000, 20)

array([7500.        , 7578.94736842, 7657.89473684, 7736.84210526,
       7815.78947368, 7894.73684211, 7973.68421053, 8052.63157895,
       8131.57894737, 8210.52631579, 8289.47368421, 8368.42105263,
       8447.36842105, 8526.31578947, 8605.26315789, 8684.21052632,
       8763.15789474, 8842.10526316, 8921.05263158, 9000.        ])

In [ ]:
x = system['时间'].values.tolist()
P = system['常驻人口总量'].values.tolist()
x = x + [2030]
P = P + [w]
P

[7869.34,
 8022.99,
 8119.81,
 8192.44,
 8281.09,
 8315.11,
 8381.47,
 8423.5,
 8446.19,
 8469.09,
 8477.26,
 9000.0]

In [272]:
data = pd.read_csv(r"E:\参加点比赛\数模研赛\data\碳排预测数据.csv")
system = data.copy()

w_list = np.linspace(7000, 10000, 20)
for i in tqdm(range(len(w_list))):
    w = w_list[i] # 权重
    sign1 = False # 判断碳达峰时间约束
    sign2 = False # 判断预测结果正负约束
    sign3 = False # 判断碳中和约束
    # 假设条件下拟合得到人口、经济与时间的关系式
    ##############################################################################################################################################
    # 对人口做拟合
    # 做多项式拟合
    x = system['时间'].values.tolist()
    P = system['常驻人口总量'].values.tolist()
    x = x + [2030]
    P = P + [w]
    def fit_p_poly(x, a, b, c):
        return a * x**2 + b * x + c
    params, convariance = curve_fit(fit_p_poly, x, P)
    a, b, c = params
    x = sp.symbols('x')
    predict_p_poly = a * x**2 + b * x + c
    # 做线性拟合
    x = system['时间'].values.tolist()
    P = system['常驻人口总量'].values.tolist()
    def fit_p_linear(x, a, b): 
        return a * x + b
    params, convariance = curve_fit(fit_p_linear, x, P)
    a, b = params
    x = sp.symbols('x')
    predict_p_linear = a * x + b 
    # 组合预测
    predict_p = 0.8 * predict_p_linear + 0.2 * predict_p_poly
    # 对经济做拟合：满足给定条件
    x = system['时间'].values.tolist()
    GDP = system['区域生产总值'].values.tolist()
    x = x + [2035, 2060]
    GDP = GDP + [2*GDP[-1], 4*GDP[-1]]
    def fit_gdp(x, a, b): # 拟合函数
        return a * x + b
    params, convariance = curve_fit(fit_gdp, x, GDP) # 将x、GDP放入fit_gdp中拟合
    a, b = params # 获取拟合参数
    x = sp.symbols('x') 
    predict_gdp = a * x + b # 获取拟合方程式


    # 多元线性回归 得到五种能源与人口、经济关系
    ##############################################################################################################################################
    # 选择自变量与应变量
    GDPP = system.loc[:, ['区域生产总值', '常驻人口总量']].values # 自变量
    EV1 = system['一次能源中化石的能源消费量'].values
    EV2 = system['一次能源中非化石的能源消费量'].values
    EV3 = system['二次能源的能源消费量'].values
    EV4 = system['化石能源发电的能源消费量'].values
    EV5 = system['非化石能源发电的能源消费量'].values

    # 求解EV1关于GDP的线性回归方程
    lr_EV1 = LinearRegression()
    lr_EV1.fit(GDPP, EV1)
    coef_EV1 = lr_EV1.coef_
    intercept_EV1 = lr_EV1.intercept_
    R2_EV1 = lr_EV1.score(GDPP, EV1)

    # 求解EV2关于GDP的线性回归方程
    lr_EV2 = LinearRegression()
    lr_EV2.fit(GDPP, EV2)
    coef_EV2 = lr_EV2.coef_
    intercept_EV2 = lr_EV2.intercept_
    R2_EV2 = lr_EV2.score(GDPP, EV2)

    # 求解EV3关于GDP的线性回归方程
    lr_EV3 = LinearRegression()
    lr_EV3.fit(GDPP, EV3)
    coef_EV3 = lr_EV3.coef_
    intercept_EV3 = lr_EV3.intercept_
    R2_EV3 = lr_EV3.score(GDPP, EV3)

    # 求解EV4关于GDP的线性回归方程
    lr_EV4 = LinearRegression()
    lr_EV4.fit(GDPP, EV4)
    coef_EV4 = lr_EV4.coef_
    intercept_EV4 = lr_EV4.intercept_
    R2_EV4 = lr_EV4.score(GDPP, EV4)

    # 求解EV5关于GDP的线性回归方程
    lr_EV5 = LinearRegression()
    lr_EV5.fit(GDPP, EV5)
    coef_EV5 = lr_EV5.coef_
    intercept_EV5 = lr_EV5.intercept_
    R2_EV5 = lr_EV5.score(GDPP, EV5)

    # 多元线性回归 得到六个部门碳排放量与五种能源消费量关系
    ##############################################################################################################################################
    # 选择自变量与应变量
    Eall = system.loc[:, ['一次能源中化石的能源消费量', '一次能源中非化石的能源消费量', '二次能源的能源消费量', '化石能源发电的能源消费量',
        '非化石能源发电的能源消费量']]
    CO2D1 = system['农林消费部门的碳排放量'].values
    CO2D2 = system['工业消费部门的碳排放量'].values
    CO2D3 = system['交通消费部门的碳排放量'].values
    CO2D4 = system['建筑消费部门的碳排放量'].values
    CO2D5 = system['居民生活消费部门的碳排放量'].values
    CO2D6 = system['能源供应部门的碳排放量'].values
    CO2 = system['碳排放量'].values

    # 求解CO2D1关于五种能源的方程
    lr_CO2D1 = LinearRegression()
    lr_CO2D1.fit(Eall, CO2D1)
    coef_CO2D1 = lr_CO2D1.coef_
    intercept_CO2D1 = lr_CO2D1.intercept_
    R2_CO2D1 = lr_CO2D1.score(Eall, CO2D1)

    # 求解CO2D2关于五种能源的方程
    lr_CO2D2 = LinearRegression()
    lr_CO2D2.fit(Eall, CO2D2)
    coef_CO2D2 = lr_CO2D2.coef_
    intercept_CO2D2 = lr_CO2D2.intercept_
    R2_CO2D2 = lr_CO2D2.score(Eall, CO2D2)

    # 求解CO2D3关于五种能源的方程
    lr_CO2D3 = LinearRegression()
    lr_CO2D3.fit(Eall, CO2D3)
    coef_CO2D3 = lr_CO2D3.coef_
    intercept_CO2D3 = lr_CO2D3.intercept_
    R2_CO2D3 = lr_CO2D3.score(Eall, CO2D3)

    # 求解CO2D4关于五种能源的方程
    lr_CO2D4 = LinearRegression()
    lr_CO2D4.fit(Eall, CO2D4)
    coef_CO2D4 = lr_CO2D4.coef_
    intercept_CO2D4 = lr_CO2D4.intercept_
    R2_CO2D4 = lr_CO2D4.score(Eall, CO2D4)

    # 求解CO2D5关于五种能源的方程
    lr_CO2D5 = LinearRegression()
    lr_CO2D5.fit(Eall, CO2D5)
    coef_CO2D5 = lr_CO2D5.coef_
    intercept_CO2D5 = lr_CO2D5.intercept_
    R2_CO2D5 = lr_CO2D5.score(Eall, CO2D5)

    # 求解CO2D6关于五种能源的方程
    lr_CO2D6 = LinearRegression()
    lr_CO2D6.fit(Eall, CO2D6)
    coef_CO2D6 = lr_CO2D6.coef_
    intercept_CO2D6 = lr_CO2D6.intercept_
    R2_CO2D6 = lr_CO2D6.score(Eall, CO2D6)

    # 计算得到CO2的方程
    ##############################################################################################################################################
    predict_EV1 = coef_EV1[0] * predict_gdp + coef_EV1[1] * predict_p + intercept_EV1
    predict_EV2 = coef_EV2[0] * predict_gdp + coef_EV2[1] * predict_p + intercept_EV2
    predict_EV3 = coef_EV3[0] * predict_gdp + coef_EV3[1] * predict_p + intercept_EV3
    predict_EV4 = coef_EV4[0] * predict_gdp + coef_EV4[1] * predict_p + intercept_EV4
    predict_EV5 = coef_EV5[0] * predict_gdp + coef_EV5[1] * predict_p + intercept_EV5

    predict_CO2D1 = coef_CO2D1[0] * predict_EV1 + \
                    coef_CO2D1[1] * predict_EV2 + \
                    coef_CO2D1[2] * predict_EV3 + \
                    coef_CO2D1[3] * predict_EV4 + \
                    coef_CO2D1[4] * predict_EV5 + intercept_CO2D1

    predict_CO2D2 = coef_CO2D2[0] * predict_EV1 + \
                    coef_CO2D2[1] * predict_EV2 + \
                    coef_CO2D2[2] * predict_EV3 + \
                    coef_CO2D2[3] * predict_EV4 + \
                    coef_CO2D2[4] * predict_EV5 + intercept_CO2D2

    predict_CO2D3 = coef_CO2D3[0] * predict_EV1 + \
                    coef_CO2D3[1] * predict_EV2 + \
                    coef_CO2D3[2] * predict_EV3 + \
                    coef_CO2D3[3] * predict_EV4 + \
                    coef_CO2D3[4] * predict_EV5 + intercept_CO2D3

    predict_CO2D4 = coef_CO2D4[0] * predict_EV1 + \
                    coef_CO2D4[1] * predict_EV2 + \
                    coef_CO2D4[2] * predict_EV3 + \
                    coef_CO2D4[3] * predict_EV4 + \
                    coef_CO2D4[4] * predict_EV5 + intercept_CO2D4

    predict_CO2D5 = coef_CO2D5[0] * predict_EV1 + \
                    coef_CO2D5[1] * predict_EV2 + \
                    coef_CO2D5[2] * predict_EV3 + \
                    coef_CO2D5[3] * predict_EV4 + \
                    coef_CO2D5[4] * predict_EV5 + intercept_CO2D5

    predict_CO2D6 = coef_CO2D6[0] * predict_EV1 + \
                    coef_CO2D6[1] * predict_EV2 + \
                    coef_CO2D6[2] * predict_EV3 + \
                    coef_CO2D6[3] * predict_EV4 + \
                    coef_CO2D6[4] * predict_EV5 + intercept_CO2D6
    predict_CO2 = predict_CO2D1 + predict_CO2D2 + predict_CO2D3 + predict_CO2D4 + predict_CO2D5 + predict_CO2D6
    # 求解碳达峰时间
    predict_CO2_d = sp.diff(predict_CO2, x)
    t1 = sp.solve(predict_CO2_d)
    t1 = int(t1[0])

    # 满足2030年碳达峰条件
    if (t1>2020) & (t1<2040):
        sign1 = True

    # 新建预测结果
    system_pre = pd.DataFrame(columns=system.columns)
    system_pre['时间'] = [i for i in range(2010, 2061)]

    for i in range(system_pre.shape[0]):
        t = system_pre.loc[i, ['时间']][0] # 获取第i个时间
        system_pre.loc[i, ['常驻人口总量']] = predict_p.subs(x, t)
        system_pre.loc[i, ['区域生产总值']] = predict_gdp.subs(x, t)
        system_pre.loc[i, ['一次能源中化石的能源消费量']] = predict_EV1.subs(x, t)
        system_pre.loc[i, ['一次能源中非化石的能源消费量']] = predict_EV2.subs(x, t)
        system_pre.loc[i, ['二次能源的能源消费量']] = predict_EV3.subs(x, t)
        system_pre.loc[i, ['化石能源发电的能源消费量']] = predict_EV4.subs(x, t)
        system_pre.loc[i, ['非化石能源发电的能源消费量']] = predict_EV5.subs(x, t)
        system_pre.loc[i, ['农林消费部门的碳排放量']] = predict_CO2D1.subs(x, t)
        system_pre.loc[i, ['工业消费部门的碳排放量']] = predict_CO2D2.subs(x, t)
        system_pre.loc[i, ['交通消费部门的碳排放量']] = predict_CO2D3.subs(x, t)
        system_pre.loc[i, ['建筑消费部门的碳排放量']] = predict_CO2D4.subs(x, t)
        system_pre.loc[i, ['居民生活消费部门的碳排放量']] = predict_CO2D5.subs(x, t)
        system_pre.loc[i, ['能源供应部门的碳排放量']] = predict_CO2D6.subs(x, t)
        system_pre.loc[i, ['碳排放量']] = predict_CO2.subs(x, t)

    # 判断是否全都为正
    has_negative_1 = (system_pre['常驻人口总量'] > 0).all()
    has_negative_2 = (system_pre['区域生产总值'] > 0).all()
    has_negative_3 = (system_pre['一次能源中化石的能源消费量'] > 0).all()
    has_negative_4 = (system_pre['一次能源中非化石的能源消费量'] > 0).all()
    has_negative_5 = (system_pre['二次能源的能源消费量'] > 0).all()
    has_negative_6 = (system_pre['化石能源发电的能源消费量'] > 0).all()
    has_negative_7 = (system_pre['非化石能源发电的能源消费量'] > 0).all()
    has_negative_8 = (system_pre['农林消费部门的碳排放量'] > 0).all()
    has_negative_9 = (system_pre['工业消费部门的碳排放量'] > 0).all()
    has_negative_10 = (system_pre['交通消费部门的碳排放量'] > 0).all()
    has_negative_11 = (system_pre['建筑消费部门的碳排放量'] > 0).all()
    has_negative_12 = (system_pre['居民生活消费部门的碳排放量'] > 0).all()
    has_negative_13 = (system_pre['能源供应部门的碳排放量'] > 0).all()
    has_negative_14 = (system_pre['碳排放量'] > 0).all()

    if has_negative_1 & has_negative_2 & has_negative_3 & has_negative_4 & has_negative_5 & has_negative_6 & has_negative_7 & \
        has_negative_8 & has_negative_9 & has_negative_10 & has_negative_11 & has_negative_12 & has_negative_13 & has_negative_14 :
        sign2 = True

    time = [2020, 2025, 2030, 2035, 2050, 2060]
    goal_2 = pd.DataFrame({'时间' : time})
    goal_2['一次能源中化石的能源消费量'] = 99
    goal_2['一次能源中非化石的能源消费量'] = 99
    goal_2['二次能源的能源消费量'] = 99
    goal_2['化石能源发电的能源消费量'] = 99
    goal_2['非化石能源发电的能源消费量'] = 99
    goal_2['碳排放量'] = 99
    for i in time:
        if i == 2020:
            goal_2.loc[goal_2['时间']==i, ['一次能源中化石的能源消费量']] = system.loc[system['时间']==i, ['一次能源中化石的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['一次能源中非化石的能源消费量']] = system.loc[system['时间']==i, ['一次能源中非化石的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['二次能源的能源消费量']] = system.loc[system['时间']==i, ['二次能源的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['化石能源发电的能源消费量']] = system.loc[system['时间']==i, ['化石能源发电的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['非化石能源发电的能源消费量']] = system.loc[system['时间']==i, ['非化石能源发电的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['碳排放量']] = system.loc[system['时间']==i, ['碳排放量']].values[0]
        else:
            goal_2.loc[goal_2['时间']==i, ['一次能源中化石的能源消费量']] = system_pre.loc[system_pre['时间']==i, ['一次能源中化石的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['一次能源中非化石的能源消费量']] = system_pre.loc[system_pre['时间']==i, ['一次能源中非化石的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['二次能源的能源消费量']] = system_pre.loc[system_pre['时间']==i, ['二次能源的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['化石能源发电的能源消费量']] = system_pre.loc[system_pre['时间']==i, ['化石能源发电的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['非化石能源发电的能源消费量']] = system_pre.loc[system_pre['时间']==i, ['非化石能源发电的能源消费量']].values[0]
            goal_2.loc[goal_2['时间']==i, ['碳排放量']] = system_pre.loc[system_pre['时间']==i, ['碳排放量']].values[0]
    goal_2['能源利用效率'] = goal_2['碳排放量'] / \
                            (goal_2['一次能源中化石的能源消费量'] + \
                            goal_2['一次能源中非化石的能源消费量'] + \
                            goal_2['二次能源的能源消费量'])
    goal_2['非化石能源消费比重'] = (goal_2['一次能源中非化石的能源消费量'] + goal_2['二次能源的能源消费量']) / \
                                (goal_2['一次能源中化石的能源消费量'] + goal_2['一次能源中非化石的能源消费量'] + goal_2['二次能源的能源消费量'] )
    if (goal_2.loc[goal_2['时间']==2060, ['非化石能源消费比重']].values[0][0] >= 0.8) & \
        (goal_2.loc[goal_2['时间']==2060, ['非化石能源消费比重']].values[0][0] <= 1):
        sign3 = True
    
    # print(sign1, sign2)
    # print(w, goal_2.loc[goal_2['时间']==2060, ['非化石能源消费比重']].values[0][0])
    if sign1:
        print("满足线性规划模型的人口={}碳达峰时间={}".format(w, t1))
        print(w, goal_2.loc[goal_2['时间']==2060, ['非化石能源消费比重']].values[0][0])


  5%|▌         | 1/20 [00:00<00:15,  1.23it/s]

满足线性规划模型的人口=7000.0碳达峰时间=2036
7000.0 0.946390394538042


 10%|█         | 2/20 [00:01<00:15,  1.13it/s]

满足线性规划模型的人口=7157.894736842105碳达峰时间=2038
7157.894736842105 0.819023221842710


100%|██████████| 20/20 [00:17<00:00,  1.12it/s]


In [ ]:
goal_2

,时间,一次能源中化石的能源消费量,一次能源中非化石的能源消费量,二次能源的能源消费量,化石能源发电的能源消费量,非化石能源发电的能源消费量,碳排放量,能源利用效率,非化石能源消费比重
0,2020,28699.99,354.01,2384,12397.72,2384,111492.9407,3.546439,0.087092
1,2025,34862.2859212458,604.403073306195,3435.07993815839,16023.6312678754,3435.07993815839,129333.213762820,3.32461009643370,0.103838029022583
2,2030,40910.7143085003,832.267437295057,4074.38584444299,19541.9970361888,4074.38584444299,146341.372487664,3.19401528687657,0.107091558066366
3,2035,48412.5990000665,1107.27623056900,4455.83866372705,24072.1155305803,4455.83866372705,166147.138168871,3.07818324541372,0.103066999821138
4,2050,79638.9909009635,2215.16918609664,4053.07859950140,43732.9873704314,4053.07859950140,242350.076950610,2.82106700967122,0.0729653039887380
5,2060,107723.867023319,3189.48663620558,2495.30645496398,61898.9988941848,2495.30645496398,307140.070920169,2.70825940990842,0.0501266224768958
